In [36]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, \
    NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D


<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


In [37]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"

df = df.sample(frac=1).reset_index(drop=True)


print(df)


# new_df = pd.read_csv(r'machine-learning-in-science-ii-2023/new_training_norm.csv')
# new_df['filename'] = new_df["image_id"].astype(str) + "_speed-" + new_df["speed"].astype(str) + "_angle-" + new_df["angle"].astype(str) + ".png"
# #new_df['filename'] = new_df["image_id"].astype(str) + ".png"

#print(new_df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# new_x_train = new_df[0:int(len(new_df) * 0.6)]
# new_x_validate = new_df[int(len(new_df) * 0.6):int(len(new_df) * 0.8)]
# new_x_evaluate = new_df[int(len(new_df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0          7813  0.5625    0.0   7813.png
1          2481  0.6250    1.0   2481.png
2         10208  0.5000    1.0  10208.png
3          1660  0.8125    1.0   1660.png
4          7457  0.6875    1.0   7457.png
...         ...     ...    ...        ...
14821      1269  0.7500    1.0   1269.png
14822      8104  0.5625    1.0   8104.png
14823      6309  0.5000    1.0   6309.png
14824      6250  0.5000    1.0   6250.png
14825      2878  0.7500    0.0   2878.png

[14826 rows x 4 columns]


In [38]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

#print(train_generator)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.


In [39]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

#transfer.trainable = False # freeze the first layers to the imagenet weights

# CNN base
model = models.Sequential()

# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

model.add(transfer)
model.build()
        
model.add(layers.Conv2D(512, (3, 3), strides= (2,2), activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

#model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),

# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))

In [40]:
model.build()


In [41]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 4, 512)         5898752   
                                                                 
 batch_normalization_11 (Bat  (None, 3, 4, 512)        2048      
 chNormalization)                                                
                                                                 
 flatten_10 (Flatten)        (None, 6144)              0         
                                                                 
 dense_11 (Dense)            (None, 64)                393280    
                                                                 
 batch_normalization_12 (Bat  (None, 64)              

In [42]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0015,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
556/556 [==============================] - 98s 171ms/step - loss: 0.0930 - val_loss: 0.0383
Epoch 2/100
556/556 [==============================] - 93s 166ms/step - loss: 0.0451 - val_loss: 0.0170
Epoch 3/100
556/556 [==============================] - 93s 167ms/step - loss: 0.0277 - val_loss: 0.0126
Epoch 4/100
556/556 [==============================] - 93s 166ms/step - loss: 0.0227 - val_loss: 0.0120
Epoch 5/100
556/556 [==============================] - 92s 166ms/step - loss: 0.0203 - val_loss: 0.0105
Epoch 6/100
556/556 [==============================] - 94s 168ms/step - loss: 0.0189 - val_loss: 0.0104
Epoch 7/100
556/556 [==============================] - 93s 168ms/step - loss: 0.0175 - val_loss: 0.0099
Epoch 8/100
556/556 [==============================] - 92s 166ms/step - loss: 0.0164 - val_loss: 0.0091
Epoch 9/100
556/556 [==============================] - 95s 171ms/step - loss: 0.0159 - val_loss: 0.0089
Epoch 10/100
556/556 [==============================] - 95s 171m

In [43]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

print(test_images)

prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.
32/32 [==============================] - 5s 123ms/step
(1020, 2)


In [49]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)


prediction = model.predict(test_generator, verbose =1)

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('evaluate.csv')



93/93 [==============================] - 31s 315ms/step - loss: 0.0114
Found 2966 validated image filenames.
52/93 [===============>..............] - ETA: 15s

: 

: 

In [45]:
diff = abs(x_evaluate["angle"] - a)

results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
results

#results.to_csv('evaluate.csv')

,Train,Predicted,Difference
11860,0.5625,0.502337,0.060163
11861,0.8750,0.739471,0.135529
11862,0.5000,0.507915,0.007915
11863,0.5000,0.488921,0.011079
11864,0.6250,0.518560,0.106440
...,...,...,...
14821,0.7500,0.731086,0.018914
14822,0.5625,0.518553,0.043947
14823,0.5000,0.489573,0.010427
14824,0.5000,0.505882,0.005882


In [46]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
results

#results.to_csv('evaluate.csv')

,Train,Predicted
11860,0.0,0.0
11861,1.0,1.0
11862,1.0,1.0
11863,1.0,1.0
11864,1.0,1.0
...,...,...
14821,1.0,1.0
14822,1.0,1.0
14823,1.0,1.0
14824,1.0,1.0


In [47]:
model.save("model.h5")

In [ ]:
def get_ML_model(model_config_path, model_weights_path, _loss_='categorical_crossentropy', _optimizer_='adam', _metrics_='accuracy'):
    # load YAML and create model
    yaml_file = open(model_config_path, 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    loaded_model = model_from_yaml(loaded_model_yaml)

    print("Model loaded")
    # load weights into new model
    loaded_model.load_weights(model_weights_path)
    print("Model_weights_loaded")
    # evaluate loaded model on test data

    loaded_model.compile(
	loss=_loss_, 
	optimizer=_optimizer_, 
	metrics=[_metrics_])    

    print("Model compiled")
    return loaded_model


model_config_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_{}\\model_{}_config.yaml".format(retrain_model_ID, model_type, retrain_model_ID)
# sub_model_weights_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_CNN_GA\\Model_{}_Checkpoints\\30R29_weights_26-0.0517-0.9803-0.1515-0.9568.hdf5".format(retrain_model_ID, retrain_model_ID)
sub_model_weights_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_{}\\Model_{}_Checkpoints\\60_weights_07-0.1071-0.9635-0.1447-0.9559.hdf5".format(retrain_model_ID, model_type, retrain_model_ID)
model = get_ML_model(model_config_file, sub_model_weights_file)
epoch_start = print("Model loaded from epoch {} - {}".format(epoch_start, sub_model_weights_file))

history = model.fit_generator(
    train_generator,
    epochs=n_epochs,
    steps_per_epoch=total_train_samples / BATCH_SIZE,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=total_validation_samples / BATCH_SIZE,
    initial_epoch=epoch_start,
    callbacks=[csv_logger, check_point, early_stopping, tensorboard])






In [ ]:
model.load("model.h5")

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0015,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)